<a href="https://colab.research.google.com/github/Aruminium/pdfConsolidation/blob/main/pdfConsolidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#インストール
%cd "/content/drive/My Drive"
!pip install PyPDF2
!pip install natsort
!apt-get install ghostscript #pdf圧縮
!pip install pycryptodome

/content/drive/My Drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ghostscript is already the newest version (9.26~dfsg+0-0ubuntu0.18.04.16).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 7.4 MB/s 


In [57]:
import PyPDF2
import subprocess
import glob
from natsort import natsorted
import os
import re
from Crypto.Cipher import AES
import unicodedata
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
#@title 準備 { vertical-output: true }

#@markdown **googleドライブのパス(基本的に変更しない)**
gdrivePath = '/content/drive/MyDrive/' #@param {type:"string"}

#@markdown **連結させたいpdfが在るディレクトリのパス**
dirPath = 'pdf/DSP/' #@param {type:"string"}

#連結するpdfをまとめたディレクトリのパス
files = glob.glob(gdrivePath + dirPath + "*.pdf")

# 全角->半角
[os.rename(pdfFile, unicodedata.normalize("NFKC", pdfFile)) for pdfFile in files]

#連結するpdfをまとめたディレクトリのパス
files = glob.glob(gdrivePath + dirPath + "*.pdf")

fileNames = [os.path.basename(pdfFile).replace(".pdf", "") for pdfFile in files]
fileNames

['DSP2022第2回:信号処理の基本+',
 'DSP2022第3回:ディジタル信号処理の基礎数学',
 'DSP2022第5回:ラプラス変換',
 'DSP2022第4回:ディジタルシステムの性質',
 'DSP2022第9回:フーリエ変換',
 'DSP2022第7回:ラプラス変換とZ変換の関係',
 'DSP2022第6回:Z_変換',
 'DSP2022第12回:ディジタル信号処理の応用(1)通信信号',
 'DSP2022第11回:ディジタルフィルタ',
 'DSP2022第10回:離散フーリエ変換',
 'DSP2022第8回:ディジタルシステムの伝達関数・周波数特性',
 'DSP2022第15回目_まとめ',
 'DSP2022第14回:ディジタル信号処理の応用(3)映像信号',
 'DSP2022第13回:ディジタル信号処理の応用(2)音声信号']

In [90]:
#@title 並び替え　{ vertical-output: true }

sorting = "ascending" #@param ["unsort", "ascending", "descending", "reversion"]

if sorting in "ascending":
    files = natsorted(files)
elif sorting in "descending":
    files = natsorted(files).reverse()
elif sorting in "reversion":
    files = files.reverse()

#確認
fileNames = [os.path.basename(pdfFile).replace(".pdf", "") for pdfFile in files]
fileNames

['DSP2022第2回:信号処理の基本+',
 'DSP2022第3回:ディジタル信号処理の基礎数学',
 'DSP2022第4回:ディジタルシステムの性質',
 'DSP2022第5回:ラプラス変換',
 'DSP2022第6回:Z_変換',
 'DSP2022第7回:ラプラス変換とZ変換の関係',
 'DSP2022第8回:ディジタルシステムの伝達関数・周波数特性',
 'DSP2022第9回:フーリエ変換',
 'DSP2022第10回:離散フーリエ変換',
 'DSP2022第11回:ディジタルフィルタ',
 'DSP2022第12回:ディジタル信号処理の応用(1)通信信号',
 'DSP2022第13回:ディジタル信号処理の応用(2)音声信号',
 'DSP2022第14回:ディジタル信号処理の応用(3)映像信号',
 'DSP2022第15回目_まとめ']

In [91]:
#@title ページ数取得　{ vertical-output: true }
pdfPages = []
sumPages = 0
for pdfFile in files:
    pdf_reader = PyPDF2.PdfFileReader(pdfFile)
    page_num = pdf_reader.getNumPages()
    pdfPages.append(sumPages)
    sumPages += page_num
    print(sumPages)
pdfPages

/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'1' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'2' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'3' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'23' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'34' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'48' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: 

50
108
175
229
279
325
369
414
456
498
540
580
632
699


/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'22' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'33' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'63' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'98' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'125' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'142' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarn

[0, 50, 108, 175, 229, 279, 325, 369, 414, 456, 498, 540, 580, 632]

In [99]:

#@title 出力　{ vertical-output: true }

#@markdown **ファイル名**
pdfName = 'DSPまとめ' #@param {type:"string"}
#@markdown **MyDrive/以下の出力path**
exportPath = 'pdf/DSP/compressioned/' #@param {type:"string"}

pdf_writer = PyPDF2.PdfFileMerger()

for file in files:
    pdf_writer.append(file)

#出力パス gdrivePath + "/ + 任意ディレクトリのpath(/のみでも大丈夫)" + fileName
outPath = gdrivePath + exportPath + pdfName + '.pdf'

# exportPathがなければ作る
if not os.path.exists(exportPath):
  os.mkdir(exportPath)

pdf_writer.write(outPath)
pdf_writer.close()

/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'1' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'2' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'3' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'23' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'34' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'48' b'0'
  PdfReadWarning,
/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1197: PdfReadWarning: 

In [100]:
#@title しおり作成 & 圧縮

#@markdown うまくいかない時は1つ前のセルのpdf結合からやり直す

writer = PyPDF2.PdfFileWriter()
reader = PyPDF2.PdfFileReader(outPath)

for i in range(reader.getNumPages()):
    #圧縮
    page = reader.getPage(i)
    page.compressContentStreams()
    writer.addPage(page)

for key, value in zip(fileNames, pdfPages):
    writer.addBookmark(key, value, parent=None)
writer.setPageMode("/UseOutlines")  # This is what tells the PDF to open to bookmarks
with open(outPath, "wb") as fp:
    writer.write(fp)